Dependencies

In [1]:
%pip install gensim
%pip install nltk

from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Pre-processing

In [26]:
# Load the corpus
corpus = LineSentence('corpus.txt')
docs = [doc for doc in corpus]
tokens = []
for doc in docs:
    tokens.extend(doc)
print(len(tokens))

# tokens = tokens[:10_000_000] # TODO: Remove
# 1_000_000 tokens é um tempo OK

# removing stop words
nltk.download('stopwords')
stop_words = stopwords.words('english')
print("Removing stopwords...")
tokens = [token for token in tokens if token not in stop_words]

# lemmatizing
print("Lemmatizing...")
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(token) for token in tokens]

# grouping into sentences
print("Grouping into sentences...")
sentences = []
sentence = []
counter = 0
words_per_sentence = 10
for token in tokens:
    counter += 1
    if counter > words_per_sentence:
        sentences.append(sentence)
        sentence = [token]
        counter = 1
        continue

    sentence.append(token)

print(len(tokens))

17005207
Removing stopwords...


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vip/victor.henrique/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Lemmatizing...
Grouping into sentences...
10890638


Word2vec model

In [24]:
print("Training model...")
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=8)

Training model...


Evaluation

In [34]:
def distanceToMostSimilarWord(input_words: list, true_word: str) -> float:
    lemmatized_words = []
    for word in input_words:
        word = lemmatizer.lemmatize(word)

        if word not in model.wv:
        lemmatized_words.append(word)


    similar_words = model.wv.most_similar(lemmatized_words)
    most_similar_word = similar_words[0][0]
    print(most_similar_word)
    return model.wv.distance(lemmatizer.lemmatize(true_word), most_similar_word)

# print(distanceToMostSimilarWord(["king"], "egg"))
# print(distanceToMostSimilarWord(["culottes"], "culottes"))

with open("questions-words.txt") as f:
    for line in f:
        if line.startswith(":"):
            continue
        words = line.split()
        print(words[:3])
        print(distanceToMostSimilarWord(words[:3], words[3]))
        break

['Athens', 'Greece', 'Baghdad']


KeyError: "Key 'Athens' not present in vocabulary"